<a href="https://colab.research.google.com/github/tphlabs/timetable_validity/blob/main/timetable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Technion courses feasibility check

Evgeny Kolonsky, 2024
v.0.1.4

Each semester a student of Technion chooses an educational track, taking number of courses out of catalogue published by the institution.
See for example recommended [plan](https://phys.technion.ac.il/images/articles/maarechet_momletzet/chart_physics_final.pdf) for physics dept.

A course has a set of groups with particular slots in a week timetable.

Any changes in catalogue or building a custom educational track may require feasibility check: whether chosen set of courses (track) interfere or not.

Here the algorythm for feasibility check is implemented.

Input data:
- courses catalogue, fetched from official source
- educational track as a list of chosen courses

Output:
- list of possible implementations
- rating of groups with 0 for groups than can not be chosen in any case.


In [1]:
import requests
import json
import numpy as np
import itertools

# full catalogue of courses fetched from SAP
# semester 2024_201 (winter 24/25)
# source code: https://github.com/michael-maltsev/technion-sap-info-fetcher
url = 'https://raw.githubusercontent.com/tphlabs/timetable_validity/main/2024_201.json'
response = requests.get(url)
data = json.loads(response.text)
print(f'Loaded {len(data)} courses')


Loaded 1058 courses


In [2]:
# display for example courses assosiated with Physics Dept
for course in data:
  course_dept = course['general']['פקולטה']
  course_name = course['general']['שם מקצוע']
  course_id = course['general']['מספר מקצוע']
  if course_dept != 'הפקולטה לפיזיקה':
    continue
  print(course_id, course_name)

01140020 מעבדה לפיסיקה 1מ'
01140021 מעבדה לפיסיקה  2מ
01140030 מעבדה לפיסיקה 2 מח'
01140032 מעבדה לפיסיקה 1ח'
01140034 מעבדה לפיסיקה 2מפ'
01140035 מעבדה לפיסיקה 3 - גלים
01140037 מעבדה לפיסיקה 4מח'
01140038 מעבדה לפיזיקה - גלים - 3מפ'
01140051 פיסיקה 1
01140052 פיסיקה 2
01140054 פיסיקה 3
01140071 פיסיקה 1מ
01140073 פיזיקה קוונטית להנדסה
01140074 פיסיקה 1פ'
01140075 פיסיקה 2ממ
01140077 פיסיקה 1ל
01140081 מעבדה לפיסיקה 1
01140082 מעבדה לפיסיקה 2
01140086 גלים
01140101 מכניקה אנליטית
01140226 דו"ח סגל מחקר סתיו
01140229 פרויקט
01140248 פיסיקה 1 ר
01140250 מעבדה לפיסיקה 5ת
01140251 מעבדה לפיסיקה 6ת
01140252 פרויקט ת
01150204 פיסיקה קוונטית 2
01160028 סמינר בפרקים נבחרים בפיסיקה חורף
01160029 מבוא לביופיסיקה
01160030 סמינר בפרקים נבחרים בפיסיקה-אביב
01160034 מערכות קוונטיות מקרוסקופיות
01160040 אינפורמציה קוונטית מתקדמת
01160083 טכנולוגיות קוונטיות
01160110 פיסיקה של האטמוספירה
01160217 פיסיקה של מצב מוצק
01160354 אסטרופיסיקה וקוסמולוגיה
01180018 תיאורית מערכות רבות גופים
01180076 מעבדה מתק

In [3]:
# course numbers of courses to be checked for compatibility
my_courses = ['01140052','01140021', '01140034']


# each course accompanied with a list of timeslots
# one timeslot takes 1, 2 or 3 hours in a day
# a group may have 1 or more timeslots in a week

# display slots for chosen courses
for course in data:
  course_id = course['general']['מספר מקצוע']
  if course_id in my_courses:
    print(course_id)
    for slot in course['schedule']:
      print(slot)

01140021
{'קבוצה': 19, 'סוג': 'מעבדה', 'יום': 'ראשון', 'שעה': '14:30 - 17:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 19}
{'קבוצה': 42, 'סוג': 'מעבדה', 'יום': 'שלישי', 'שעה': '15:30 - 18:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 42}
{'קבוצה': 43, 'סוג': 'מעבדה', 'יום': 'שלישי', 'שעה': '15:30 - 18:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 43}
{'קבוצה': 54, 'סוג': 'מעבדה', 'יום': 'רביעי', 'שעה': '14:30 - 17:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 54}
{'קבוצה': 55, 'סוג': 'מעבדה', 'יום': 'רביעי', 'שעה': '14:30 - 17:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 55}
{'קבוצה': 63, 'סוג': 'מעבדה', 'יום': 'חמישי', 'שעה': '11:30 - 14:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 63}
01140034
{'קבוצה': 42, 'סוג': 'מעבדה', 'יום': 'שלישי', 'שעה': '15:30 - 18:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 42}
{'קבוצה': 42, 'סוג': 'פרויקט', 'יום': 'רביעי', 'שעה': '14:30 - 17:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 42}
01140

In [4]:
# function to make hoursmap as 2d np array days x hours
# we read day and hours out of text description in timeslot
# suggested that all timeslots start and xx:30, between 08:30 and 19:30,
# which is not quite so for all published courses (sports may be exception)

WEEK = ['ראשון','שני','שלישי','רביעי','חמישי']
#SLOTS = [f'{h:02d}:30' for  h in range(8, 19)] # 1 hour slots
SLOTS = '\t'.join([f'{h:02d}:00\t{h:02d}:30' for  h in range(8, 19)]).split('\t') # 1/2 hour slots

def EMPTY_MAP():
  return np.zeros((len(WEEK), len(SLOTS)), dtype=int)

def parse_slot(slot):
  group_type = slot['סוג']
  group_number = slot['קבוצה']
  group_hour = slot['שעה']
  group_day = slot['יום']
  hours_map = EMPTY_MAP()
  hours_from, hours_to = group_hour.split(' - ')
  if hours_from not in SLOTS or hours_to not in SLOTS:
    # error message: irregular hours
    return 0
  hours_from_ind, hours_to_ind = SLOTS.index(hours_from), SLOTS.index(hours_to)
  days_ind = WEEK.index(group_day)
  hours_map[days_ind, hours_from_ind:hours_to_ind] = 1
  return hours_map


In [5]:
# just an example to test the algorythm: lets read one slo
slots = [
      {
        "קבוצה": 14,        "סוג": "הרצאה",        "יום": "רביעי",        "שעה": "10:30 - 12:30",        "בניין": "",        "חדר": 0,        "מרצה/מתרגל": "ד\"ר אירנה גורליק",        "מס.": 10      },
       {
        "קבוצה": 14,        "סוג": "תרגול",        "יום": "חמישי",        "שעה": "08:30 - 10:30",        "בניין": "",        "חדר": 0,        "מרצה/מתרגל": "",        "מס.": 14      },
      {
        "קבוצה": 14,        "סוג": "תרגול",        "יום": "ראשון",        "שעה": "11:30 - 12:30",        "בניין": "",        "חדר": 0,        "מרצה/מתרגל": "",        "מס.": 14      },
      {
        "קבוצה": 77,        "סוג": "תרגול",        "יום": "ראשון",        "שעה": "16:00 - 17:00",        "בניין": "",        "חדר": 0,        "מרצה/מתרגל": "",        "מס.": 77
      }
    ]
print(slots[-1])
print(SLOTS)
print(parse_slot(slots[-1]))


{'קבוצה': 77, 'סוג': 'תרגול', 'יום': 'ראשון', 'שעה': '16:00 - 17:00', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 77}
['08:00', '08:30', '09:00', '09:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30', '13:00', '13:30', '14:00', '14:30', '15:00', '15:30', '16:00', '16:30', '17:00', '17:30', '18:00', '18:30']
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [6]:
# one more test:
# let's aggregate hour map for group 12, having 3 slots in a week
hours_map = EMPTY_MAP()
for slot in slots:
  if slot['קבוצה'] == 14:
    hours_map += parse_slot(slot)
    print(slot)
print(hours_map)

{'קבוצה': 14, 'סוג': 'הרצאה', 'יום': 'רביעי', 'שעה': '10:30 - 12:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': 'ד"ר אירנה גורליק', 'מס.': 10}
{'קבוצה': 14, 'סוג': 'תרגול', 'יום': 'חמישי', 'שעה': '08:30 - 10:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 14}
{'קבוצה': 14, 'סוג': 'תרגול', 'יום': 'ראשון', 'שעה': '11:30 - 12:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 14}
[[0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [7]:
semester1 = ['02340128', '01040031','01040064','01140074','01140020'] #, '03940820'] #, '00440102'
semester2 = ['03240033', '01040032', '01040013', '01140076', '01140021']
my_courses = semester1

In [8]:
# activity is a combination of course and group
# here we build a hourmap for each activity
# and display result

exclude_groups = [77, 86] # chinese and inernational groups

activity = {}

for course in data:
  course_id = course['general']['מספר מקצוע']
  if course_id not in my_courses:
    continue

  activity[course_id] = {}
  slots = course['schedule']
  for slot in slots:
    group_id = slot['קבוצה']

    if group_id in exclude_groups:
      continue

    # aggregate hour maps for groups as lecture/practice
    # having more than one timeslot in a week
    activity[course_id][group_id] = \
      activity[course_id].get(group_id, EMPTY_MAP()) + parse_slot(slot)
    #if key not in activity[course_id]:
    #  activity[course_id][key] = parse_slot(slot)
    #else:
    #  activity[course_id][key] += parse_slot(slot)

# display activities with their hourmaps
for course_id in activity:
  for group_id in activity[course_id]:
    print(course_id, group_id)
    print(activity[course_id][group_id])



01040031 21
[[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]]
01040031 22
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]]
01040031 23
[[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]]
01040031 40
[[0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
01040031

In [9]:
# here is the cell  where all possible combination of groups for each activity are created
# and checked against feasibility by aggregating their hour maps
# feasible combination has only ones and zeroes in aggregated hours map
# 2 and more means activities overlapping

# list of iterators: for each course all groups
combins = [
            [f'{course}_{group}' for group in activity[course]]
            for course in activity.keys()
         ]


rating = {} # group rating counter

print('Feasible combinations:')
for combination in itertools.product(*combins):
    # init hourmap with zeroes
    check = EMPTY_MAP()

    for group in combination:
      # init counter if met 1st time
      rating[group] = rating.get(group, 0)
      # extract course and group ids from element in combinations
      course_id, group_id = group.split('_')
      # aggregate hours for groups in combination
      check += activity[course_id][int(group_id)]
    # check aggregated hourmap against overload criterion
    if np.sum(check > 1) > 0: #
      continue
    # print out combination that meets feasibility criteria
    print(combination)
    #print(check)

    # aggregate rating counter
    for group in combination:
      rating[group] += 1

print('Group rating:')
for group in sorted(rating):
  print(f'{group}: {rating[group]}')

Feasible combinations:
('01040031_40', '01040064_21', '01140020_14', '01140074_13', '02340128_21')
('01040031_40', '01040064_21', '01140020_14', '01140074_13', '02340128_23')
('01040031_40', '01040064_21', '01140020_14', '01140074_13', '02340128_24')
('01040031_40', '01040064_21', '01140020_14', '01140074_13', '02340128_32')
('01040031_40', '01040064_21', '01140020_14', '01140074_13', '02340128_41')
('01040031_40', '01040064_21', '01140020_14', '01140074_13', '02340128_42')
('01040031_40', '01040064_21', '01140020_14', '01140074_13', '02340128_43')
('01040031_40', '01040064_21', '01140020_14', '01140074_14', '02340128_11')
('01040031_40', '01040064_21', '01140020_14', '01140074_14', '02340128_12')
('01040031_40', '01040064_21', '01140020_14', '01140074_14', '02340128_13')
('01040031_40', '01040064_21', '01140020_14', '01140074_14', '02340128_21')
('01040031_40', '01040064_21', '01140020_14', '01140074_14', '02340128_23')
('01040031_40', '01040064_21', '01140020_14', '01140074_14', '023

In [10]:
# display for example courses assosiated with Physics Dept
for course in data:
  course_dept = course['general']['פקולטה']
  course_name = course['general']['שם מקצוע']
  course_id = course['general']['מספר מקצוע']
  if course_id == '01040031':
    shedule = course['schedule']
    for slot in shedule:
      print(slot)


{'קבוצה': 21, 'סוג': 'תרגול', 'יום': 'רביעי', 'שעה': '08:30 - 10:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 21}
{'קבוצה': 21, 'סוג': 'תרגול', 'יום': 'ראשון', 'שעה': '14:30 - 15:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 21}
{'קבוצה': 21, 'סוג': 'הרצאה', 'יום': 'שלישי', 'שעה': '08:30 - 10:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 20}
{'קבוצה': 21, 'סוג': 'הרצאה', 'יום': 'חמישי', 'שעה': '10:30 - 12:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 20}
{'קבוצה': 22, 'סוג': 'תרגול', 'יום': 'שלישי', 'שעה': '14:30 - 15:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 22}
{'קבוצה': 22, 'סוג': 'תרגול', 'יום': 'חמישי', 'שעה': '08:30 - 10:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 22}
{'קבוצה': 22, 'סוג': 'הרצאה', 'יום': 'שלישי', 'שעה': '08:30 - 10:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 20}
{'קבוצה': 22, 'סוג': 'הרצאה', 'יום': 'חמישי', 'שעה': '10:30 - 12:30', 'בניין': '', 'חדר': 0, 'מרצה/מתרגל': '', 'מס.': 20}
{'קבוצה': 23, 'סוג': 'תר